## Форматирование строк

In [74]:
import pandas as pd
import numpy as np

In [2]:
recipes = pd.read_csv("./src/tables/recipes_sample_6.csv")

---

1. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
|      id      |  minutes  |
|--------------|-----------|
|    61178     |    65     |
|    202352    |    80     |
|    364322    |    150    |
|    26177     |    20     |
|    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [3]:
recipes.sample(5)[['id', 'minutes']].style.hide(axis="index")

id,minutes
335969,50
511448,15
37842,60
88861,80
179462,30


---
2. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [4]:
from lxml import etree
with open("./src/tables/steps_sample.xml", 'rb') as f:
    data = etree.fromstring(f.read())

steps = {}
for i in data:
    steps[i.find('id').text] = list(map(lambda x: x.text, i.find('steps')))


In [5]:
def show_info(recipes, steps, ind):
    row = recipes.loc[recipes['id'] == ind]
    name, author, minutes = row[['name', 'contributor_id', 'minutes']].values[0]
    steps = '\n'.join(f"{n+1}. {t.capitalize()}" for n, t in enumerate(steps[str(ind)]))

    return name.title() + '\n'*2 + steps + '\n----------\nАвтор: ' +\
    str(author) + '\nСреднее время приготовления: ' + str(minutes) + ' минут'



print(show_info(recipes, steps, 170895))

Leeks And Parsnips  Sauteed Or Creamed

1. Clean the leeks and discard the dark green portions
2. Cut the leeks lengthwise then into one-inch pieces
3. Melt the butter in a medium skillet , med
4. Heat
5. Add the garlic and fry 'til fragrant
6. Add leeks and fry until the leeks are tender , about 6-minutes
7. Meanwhile , peel and chunk the parsnips into one-inch pieces
8. Place in a steaming basket and steam 'til they are as tender as you prefer
9. I like them fork-tender
10. Drain parsnips and add to the skillet with the leeks
11. Add salt and pepper
12. Gently sautee together for 5-minutes
13. At this point you can serve it , or continue on and cream it:
14. In a jar with a screw top , add the half-n-half and arrowroot
15. Shake 'til blended
16. Turn heat to low under the leeks and parsnips
17. Pour in the arrowroot mixture , stirring gently as you pour
18. If too thick , gradually add the water
19. Let simmer for a couple of minutes
20. Taste to adjust seasoning , probably an additi

---
## Работа с регулярными выражениями

---
3. Напишите регулярное выражение, которое ищет следующий паттерн в строке:

число(1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes.

Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082.\
Выведите на экран все непустые результаты, найденные по данному шаблону.


In [37]:
import re


pattern = r"\d+ (?:hours?|minutes?)"
for step in steps['25082']:
    res = re.findall(pattern, step)
    if res:
        print(*res, sep='\n')

20 minutes
10 minutes
2 hours
10 minutes
20 minutes
30 minutes


---
4. Напишите регулярное выражение, которое ищет шаблон вида `this..., but` в начале строки. Между словом `this` и частью `, but` может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом `but` может присутствовать или отсутствовать.

Используя строковые методы pd.Series, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [35]:
pattern = r'^this[\w _]*, ?but'
mask = recipes['description'].str.contains(pattern)
by_pattern = recipes[mask.fillna(False)]
print(len(by_pattern))
print(by_pattern.head(3)['description'].values)
by_pattern.head(3)

134
["this is a great meal eaten the same day ,but even better the next day , if you can wait! add your favourite spices, but try it first as it is and i think that you will enjoy the 'vegetable' taste. good for freezing."
 "this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers."
 'this is kind of similar to some of the other versions out there, but it is the best and easiest i have found']


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
76,neezy peazy lunch,24760,35,29291,2002-04-08,NaN,"this is a great meal eaten the same day ,but e...",6.0
183,3 pepper quiche,311313,75,186802,2008-07-02,NaN,this was adapted from a recipe i found on the ...,13.0
337,addictive chex mix,277562,21,260464,2008-01-08,NaN,this is kind of similar to some of the other v...,12.0


---
5. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [59]:
new = []
for step in steps['72367']:
    new.append(re.sub(r'(\d+) / (\d+)', r'\1/\2', step))
steps['72367'] = new
print(*steps['72367'], sep='\n')


mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


---
## Сегментация текста

---
6. Разбейте тексты шагов рецептов на слова при помощи пакета nltk. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов(для проверки можно воспользоваться str.isalpha). При подсчете количества уникальных слов не учитывайте регистр.


In [73]:
import nltk

words = set()
for recipe in steps.values():
    for step in recipe:
        for word in filter(str.isalpha, nltk.word_tokenize(step.lower())):
            words.add(word)

print(len(words))


14926


---
7. Разбейте описания рецептов из recipes на предложения при помощи пакета nltk. Найдите 5 самых длинных описаний(по количеству предложений) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.


In [98]:
def f(data):
    data = str(data)
    if not data:
        return 0
    return len(nltk.sent_tokenize(data))


mask = np.argsort(recipes['description'].apply(f)[::-1])
recipes.loc[mask.values].head(5)


,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
22090,rainforest trail mix,431047,3,542159,2010-06-26,3.0,a nice quick snack to keep on hand. also a goo...,7.0
14999,jicama mango salad with mozzarella chilies,81253,15,4470,2004-01-16,2.0,jicama salads are so refreshing and crunchy. w...,9.0
19831,paprika pork chops,517573,40,204024,2014-07-31,12.0,recipe from ruscuisine.com,NaN
19830,paprika oven roasted potatoes,185101,40,53932,2006-09-07,18.0,from everybody's san francisco cookbook.,NaN
19828,paprika chicken in yoghurt,12082,50,16140,2001-09-25,NaN,a simple chicken dish lovely for a midweek sup...,NaN


---
8. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
```
Для определения части речи слова можно воспользоваться nltk.pos_tag.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.

In [107]:
def parts_of_speech(sent):
    r1 = ''
    r2 = ''
    # for word in sent.split():
    for word, ps in nltk.pos_tag(sent.split()):
        r2 += word + ' '
        r1 += f'{ps:^{len(word)+1}}'
    print(r1)
    print(r2)

parts_of_speech(recipes[recipes['id'] == 241106].name.values[0])


   JJ      NNS   IN     NNS     VBP   JJ    CC   JJ    NNS  
eggplant steaks with chickpeas feta cheese and black olives 
